<a href="https://colab.research.google.com/github/IvanStukalov/MLTechnologies/blob/rk-2/%D0%98%D0%A35_64%D0%91_%D0%A1%D1%82%D1%83%D0%BA%D0%B0%D0%BB%D0%BE%D0%B2_%D0%A0%D0%9A2_%D0%A2%D0%9C%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ИУ5-64Б Стукалов РК-2 ТМО

## вариант 15

Импортируем библиотеки

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

Импортируем [датасет](https://www.kaggle.com/noriuk/us-education-datasets-unification-project) (states_all_extended)

In [32]:
df=pd.read_csv("/content/states_all_extended.csv")


In [33]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,G08_HI_A_READING,G08_HI_A_MATHEMATICS,G08_AS_A_READING,G08_AS_A_MATHEMATICS,G08_AM_A_READING,G08_AM_A_MATHEMATICS,G08_HP_A_READING,G08_HP_A_MATHEMATICS,G08_TR_A_READING,G08_TR_A_MATHEMATICS
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Преобразуем категориальные переменные в числовые, если это необходимо
Например, используем one-hot encoding для категориальных переменных

In [35]:
df = pd.get_dummies(df)

Обработаем пустые значения - вставим на их место среднее

In [36]:
df = df.fillna(df.mean())

Нормализация числовых признаков


In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_features] = scaler.fit_transform(df[numerical_features])


Разделение на обучающую и тестовую выборки

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('target_column', axis=1)
y = df['target_column']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Построение моделей

### Линейная регрессия

In [40]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)


### Градиентный бустинг (например, LightGBM)


In [42]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_eval = lgb.Dataset(X_test, label=y_test, free_raw_data=False)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'rmse'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval)
predictions_gbm = gbm.predict(X_test, num_iteration=gbm.best_iteration)


### Оценка моделей


Для линейной регрессии и градиентого бустинга можно использовать коэффициент детерминации (R^2) и среднеквадратичную ошибку (RMSE).

Выбрали эти метрики потому что (R^2) и RMSE (корень из среднеквадратической ошибки) для оценки качества моделей регрессии предоставляют разные аспекты информации о производительности модели.

(R^2) (коэффициент детерминации) измеряет долю дисперсии зависимой переменной, объясняемую независимыми переменными в модели.
RMSE представляет собой стандартное отклонение ошибок прогнозирования, то есть квадратный корень из среднеквадратической ошибки (MSE). RMSE предпочтительнее MSE для интерпретации, поскольку имеет те же единицы измерения

In [43]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

r2_lr = r2_score(y_test, predictions_lr)
mse_lr = mean_squared_error(y_test, predictions_lr)
rmse_lr = np.sqrt(mse_lr)

print(f"Linear Regression R^2: {r2_lr}, RMSE: {rmse_lr}")


Linear Regression R^2: 0.9999999999999925, RMSE: 0.4320008583978697


In [44]:
r2_gbm = r2_score(y_test, predictions_gbm)
mse_gbm = mean_squared_error(y_test, predictions_gbm)
rmse_gbm = np.sqrt(mse_gbm)

print(f"Gradient Boosting R^2: {r2_gbm}, RMSE: {rmse_gbm}")


Gradient Boosting R^2: 0.827132507732822, RMSE: 2060356.8333702348


### Вывод:

Предсказания линейной регрессии получились лучше, чем предсказания градиентного бустинга.

(R^2) показывает, какая часть дисперсии целевой переменной объясняется моделью. Значение 0.9999999999999925 для линейной регрессии говорит о том, что модель очень хорошо объясняет данные

RMSE показывает среднюю величину ошибок прогнозирования. Значение 2060356 для градиентного бустинга указывает на то, что модель делает ошибки прогнозирования, что может быть связано с переобучением